3 - Train and validate a model to predict length of stay. For this task you will need to choose a window size to train your data (too many days will delay decisions about patients when the system is deployed, too few days will probably produce a very shortsighted predictor. Choose with care).

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.metrics import mean_absolute_error


# Load  MIMIC-III tables
base_path = 'C:/Users/Ines/Desktop/Nova_entrega/Parte_topicos/'

icustays_cleaned = pd.read_excel(f'{base_path}icustays_cleaned.xlsx')
diagnoses_icd_cleaned = pd.read_excel(f'{base_path}diagnoses_icd_cleaned.xlsx')
chartevents = pd.read_csv(f'{base_path}CHARTEVENTS.csv')

C:\Users\Ines\AppData\Local\Temp\ipykernel_868\3242706356.py:30: DtypeWarning: Columns (8,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  chartevents = pd.read_csv(f'{base_path}CHARTEVENTS.csv')


In [2]:
chartevents.info(verbose=True, show_counts=True)
chartevents.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39195630 entries, 0 to 39195629
Data columns (total 15 columns):
 #   Column        Non-Null Count     Dtype  
---  ------        --------------     -----  
 0   ROW_ID        39195630 non-null  int64  
 1   SUBJECT_ID    39195630 non-null  int64  
 2   HADM_ID       39195630 non-null  int64  
 3   ICUSTAY_ID    39161239 non-null  float64
 4   ITEMID        39195630 non-null  int64  
 5   CHARTTIME     39195630 non-null  object 
 6   STORETIME     32882957 non-null  object 
 7   CGID          32882957 non-null  float64
 8   VALUE         38919020 non-null  object 
 9   VALUENUM      36320500 non-null  float64
 10  VALUEUOM      34093161 non-null  object 
 11  WARNING       34303752 non-null  float64
 12  ERROR         34303752 non-null  float64
 13  RESULTSTATUS  289761 non-null    object 
 14  STOPPED       4864645 non-null   object 
dtypes: float64(5), int64(4), object(6)
memory usage: 4.4+ GB


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED
0,788,36,165660,241249.0,223834,2134-05-12 12:00:00,2134-05-12 13:56:00,17525.0,15.0,15.00,L/min,0.0,0.0,NaN,NaN
1,789,36,165660,241249.0,223835,2134-05-12 12:00:00,2134-05-12 13:56:00,17525.0,100.0,100.00,NaN,0.0,0.0,NaN,NaN
2,790,36,165660,241249.0,224328,2134-05-12 12:00:00,2134-05-12 12:18:00,20823.0,0.37,0.37,NaN,0.0,0.0,NaN,NaN
3,791,36,165660,241249.0,224329,2134-05-12 12:00:00,2134-05-12 12:19:00,20823.0,6.0,6.00,min,0.0,0.0,NaN,NaN
4,792,36,165660,241249.0,224330,2134-05-12 12:00:00,2134-05-12 12:19:00,20823.0,2.5,2.50,NaN,0.0,0.0,NaN,NaN


In [3]:
chartevents_cleaned = chartevents[chartevents['ICUSTAY_ID'].notna()]

This code filters out rows from the DataFrame where the ICUSTAY_ID column has null (missing) values, ensuring that only records with valid ICU stay identifiers are kept for analysis.

In [4]:
chartevents_cleaned['ICUSTAY_ID'] = chartevents_cleaned['ICUSTAY_ID'].astype(int)
icustays_cleaned['ICUSTAY_ID'] = icustays_cleaned['ICUSTAY_ID'].astype(int)

chartevents_cleaned= chartevents_cleaned[chartevents_cleaned['ICUSTAY_ID'].isin(icustays_cleaned['ICUSTAY_ID'])]

print(f"ICUSTAY_IDs únicos antes do filtro: {chartevents['ICUSTAY_ID'].nunique()}")
print(f"ICUSTAY_IDs únicos após filtro: {chartevents_cleaned['ICUSTAY_ID'].nunique()}")

C:\Users\Ines\AppData\Local\Temp\ipykernel_868\2818262811.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chartevents_cleaned['ICUSTAY_ID'] = chartevents_cleaned['ICUSTAY_ID'].astype(int)


ICUSTAY_IDs únicos antes do filtro: 27241
ICUSTAY_IDs únicos após filtro: 27241


This code is aimed at better understanding the chartevents dataset by ensuring consistency with the icustays dataset. It converts the ICUSTAY_ID columns in both datasets to integers to avoid type mismatches, then filters chartevents_cleaned to keep only records whose ICUSTAY_ID exists in icustays_cleaned. Finally, it compares and prints the number of unique ICU stay IDs before and after this filtering to show how many events remain linked to valid ICU stays.

In [5]:
unique_ids_in_icustays = set(icustays_cleaned['ICUSTAY_ID'].unique())
print(f"Total ICUSTAY_IDs in icustays_cleaned: {len(unique_ids_in_icustays)}")
print(len(icustays_cleaned))

Total ICUSTAY_IDs in icustays_cleaned: 61522
61522


Confirming that every line have a unique 'ICUSTAY_ID'.

In [6]:
chartevents_cleaned = chartevents_cleaned.drop(columns=['ROW_ID','STORETIME', 'CGID', 'VALUENUM','VALUEUOM','WARNING', 'ERROR', 'RESULTSTATUS', 'STOPPED'])

In [7]:
#Filter diagnoses_icd_cleaned where recode == '6'
hadm_ids_icd6 = diagnoses_icd_cleaned[diagnoses_icd_cleaned['recode'] == 6]['HADM_ID'].unique()

#Get the ICUSTAY_IDs corresponding to these HADM_IDs
icustays_ids_icd6 = icustays_cleaned[icustays_cleaned['HADM_ID'].isin(hadm_ids_icd6)]['ICUSTAY_ID'].unique()

#Filter chartevents based on these ICUSTAY_IDs
chartevents_icd6 = chartevents_cleaned[chartevents_cleaned['ICUSTAY_ID'].isin(icustays_ids_icd6)]

print(f"recode == '6' → HADM_IDs: {len(hadm_ids_icd6)}")
print(f"Corresponding ICUSTAY_IDs: {len(icustays_ids_icd6)}")
print(f"Filtered records in chartevents: {len(chartevents_icd6)}")

# Filter chartevents_icd6 to keep only these ICUSTAY_IDs
chartevents_icd6 = chartevents_icd6[chartevents_icd6['ICUSTAY_ID'].isin(unique_ids_in_icustays)]
print(f"Filtered records in chartevents: {len(chartevents_icd6)}")


recode == '6' → HADM_IDs: 43243
Corresponding ICUSTAY_IDs: 45795
Filtered records in chartevents: 34586640
Filtered records in chartevents: 34586640


This code filters the clinical data to focus on patients with a specific diagnosis category (recode == 6). First, it extracts hospital admission IDs (HADM_ID) for diagnoses with recode 6. Then, it finds the corresponding ICU stay IDs (ICUSTAY_ID) linked to those admissions. Using these ICU stay IDs, it filters the chartevents_cleaned dataset to keep only records related to these patients. Finally, it further filters to include only ICU stays present in a predefined list (unique_ids_in_icustays). The printed counts show the number of relevant admissions, ICU stays, and filtered clinical event records at each step.

In [8]:
# Filter rows where itemid is a string
chartevents_str_itemid = chartevents_icd6[chartevents_icd6['VALUE'].apply(lambda x: isinstance(x, str))]

# View the unique values of these strings
print(chartevents_str_itemid['VALUE'].unique())

chartevents_icd6['VALUE'] = pd.to_numeric(chartevents_icd6['VALUE'], errors='coerce').astype('float64')

# Filter rows where itemid is a string
chartevents_str_itemid = chartevents_icd6[chartevents_icd6['VALUE'].apply(lambda x: isinstance(x, str))]

# View the unique values of these strings
print(chartevents_str_itemid['VALUE'].unique())


['91.4' '0' '170' ... '506.12200927734375' '921.9329833984375'
 '12.06089973449707']
[]


This code first filters chartevents_icd6 to find rows where the VALUE column contains string data, displaying the unique string values found. Then, it attempts to convert all VALUE entries to numeric values, coercing any non-convertible entries to NaN. After this conversion, it checks again for any remaining string values in VALUE and prints their unique occurrences, helping to identify and handle non-numeric data within this column.

In [9]:
print((chartevents_icd6['VALUE'] == 0).any())

mean_values = chartevents_icd6.groupby('ITEMID')['VALUE'].mean()

def fill_value(row):
    if pd.isna(row['VALUE']):
        return mean_values.get(row['ITEMID'], np.nan)  
    else:
        return row['VALUE']
def fill_value(row):
    if pd.isna(row['VALUE']):
        return mean_values.get(row['ITEMID'], np.nan) 
    else:
        return row['VALUE']

chartevents_icd6['VALUE'] = chartevents_icd6['VALUE'].fillna(chartevents_icd6['ITEMID'].map(mean_values))    
print(chartevents_icd6['VALUE'].isna().sum())

True
2000537


This code first checks if there are any zero values in the VALUE column of chartevents_icd6. Then, it calculates the mean VALUE for each ITEMID. Next, it defines a function fill_value to replace missing (NaN) values in VALUE with the corresponding ITEMID mean. However, instead of using this function explicitly, it uses pandas' fillna combined with map to directly fill NaN values with the mean for each ITEMID. Finally, it prints the count of remaining missing values in VALUE after this imputation, showing how many NaNs remain.

In [10]:
print(f"ICUSTAY_IDs únicos: {chartevents_icd6['ICUSTAY_ID'].nunique()}")
print(f"ITEMIDs únicos: {chartevents_icd6['ITEMID'].nunique()}")

ICUSTAY_IDs únicos: 23535
ITEMIDs únicos: 1578


In [11]:
print(chartevents_icd6.dtypes)


SUBJECT_ID      int64
HADM_ID         int64
ICUSTAY_ID      int64
ITEMID          int64
CHARTTIME      object
VALUE         float64
dtype: object


In [12]:
df_grouped = chartevents_icd6.groupby(['ICUSTAY_ID', 'ITEMID'])['VALUE'].mean()

print(df_grouped.head(20))

df_pivot = df_grouped.unstack()

print(df_pivot.head())
print(len(df_pivot))

ICUSTAY_ID  ITEMID
200001      220045     84.616162
            220046    120.000000
            220047     58.571429
            220050    109.600000
            220051     54.184615
            220052     83.686567
            220056     82.500000
            220058    147.500000
            220179    102.129032
            220180     56.064516
            220181     66.870968
            220210     20.535354
            220224     95.000000
            220227     92.000000
            220228      7.766667
            220235     42.428571
            220277     98.354167
            220545     24.600000
            220546      3.233333
            220587     29.000000
Name: VALUE, dtype: float64
ITEMID      1       2       3       24      25      26      27      28      \
ICUSTAY_ID                                                                   
200001         NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
200010         NaN     NaN     NaN     NaN     NaN     NaN 

This code groups the chartevents_icd6 data by ICUSTAY_ID and ITEMID, calculating the mean VALUE for each combination. It then prints the first 20 grouped results. Next, it reshapes the grouped data into a pivot table where each row represents an ICU stay (ICUSTAY_ID), each column corresponds to an ITEMID, and the cells contain the average values for those items during that stay. Finally, it prints the first few rows of this pivot table and the total number of ICU stays (rows) in the pivoted dataset.

In [13]:
nunique_per_column = df_pivot.nunique(dropna=True)

cols_to_keep = nunique_per_column[nunique_per_column > 1].index

df_pivot_filtered = df_pivot[cols_to_keep]

print(f"Columns before: {df_pivot.shape[1]}")
print(f"Columns after removing constant ones: {df_pivot_filtered.shape[1]}")

Columns before: 1578
Columns after removing constant ones: 852


This code counts the number of unique (non-NaN) values in each column of the pivoted DataFrame df_pivot. It then selects only the columns that have more than one unique value—effectively removing columns that are constant (the same value for all ICU stays). The filtered DataFrame df_pivot_filtered keeps only these variable columns. Finally, it prints the number of columns before and after this filtering to show how many constant columns were removed.

In [14]:
df_pivot_filtered = df_pivot_filtered.fillna(df_pivot_filtered.mean())
df_pivot_filtered.head()

ITEMID,2,3,25,26,29,50,52,55,59,60,...,228375,228376,228377,228378,228379,228380,228381,228382,228392,228444
ICUSTAY_ID,,,,,,,,,,,,,,,,,,,,,
200001,0.985331,0.955861,164.428793,259.747692,-35.527874,20.411955,80.727524,94.019236,109.490318,3.618011,...,35.437574,17.198265,10.726075,-0.431593,1.086753,65.633333,1045.946411,1919.428812,25.911111,40.935808
200010,0.985331,0.955861,164.428793,259.747692,-35.527874,20.411955,80.727524,94.019236,109.490318,3.618011,...,35.437574,17.198265,10.726075,-0.431593,1.086753,65.633333,1045.946411,1919.428812,25.911111,40.935808
200011,0.985331,0.955861,164.428793,259.747692,-35.527874,20.411955,80.727524,94.019236,109.490318,3.618011,...,35.437574,17.198265,10.726075,-0.431593,1.086753,65.633333,1045.946411,1919.428812,25.911111,40.935808
200016,0.985331,0.955861,164.428793,259.747692,-35.527874,20.411955,80.727524,94.019236,109.490318,3.618011,...,35.437574,17.198265,10.726075,-0.431593,1.086753,65.633333,1045.946411,1919.428812,25.911111,40.935808
200021,0.985331,0.955861,164.428793,259.747692,-35.527874,20.411955,80.727524,94.019236,109.490318,3.618011,...,35.437574,17.198265,10.726075,-0.431593,1.086753,65.633333,1045.946411,1919.428812,25.911111,40.935808


In [15]:
df = df_pivot_filtered.merge(icustays_cleaned[['ICUSTAY_ID', 'LOS']], on='ICUSTAY_ID', how='inner')
print(len(df))

23535


Considering 24h:

In [16]:
chartevents_icd6['CHARTTIME'] = pd.to_datetime(chartevents_icd6['CHARTTIME'], format='mixed', errors='coerce')
icustays_cleaned['INTIME'] = pd.to_datetime(icustays_cleaned['INTIME'])

chartevents_icd6['CHARTTIME']= chartevents_icd6['CHARTTIME'].dt.floor('min')
icustays_cleaned['INTIME']= icustays_cleaned['INTIME'].dt.floor('min')

df_merged = chartevents_icd6.merge(
    icustays_cleaned[['ICUSTAY_ID', 'INTIME']],
    on='ICUSTAY_ID',
    how='left'
)

# Filter for observations within the first 24 hours
df_merged = df_merged[df_merged['CHARTTIME'] <= df_merged['INTIME'] + pd.Timedelta(hours=24)]
print(df_merged.columns)

# Filter df_outro to keep only the icustay_id values that are in df_merged
df = df[df['ICUSTAY_ID'].isin(df_merged['ICUSTAY_ID'].unique())]
print(len(df))


Index(['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'ITEMID', 'CHARTTIME', 'VALUE',
       'INTIME'],
      dtype='object')
23492


In [17]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split X and y
X = df.drop(columns=['LOS', 'ICUSTAY_ID']) 
y = df['LOS']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [18]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

# Predictions
y_pred = model.predict(X_test_scaled)

# Evaluation
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"MAE: {mae:.2f}")   
print(f"RMSE: {rmse:.2f}")


MAE: 1.56
RMSE: 3.24


Considering 12h:

In [19]:
df_merged = df_merged[df_merged['CHARTTIME'] <= df_merged['INTIME'] + pd.Timedelta(hours=12)]
print(df_merged.columns)

df = df[df['ICUSTAY_ID'].isin(df_merged['ICUSTAY_ID'].unique())]
print(len(df))

Index(['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'ITEMID', 'CHARTTIME', 'VALUE',
       'INTIME'],
      dtype='object')
23413


In [20]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = df.drop(columns=['LOS', 'ICUSTAY_ID']) 
y = df['LOS']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [21]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"MAE: {mae:.2f}")   
print(f"RMSE: {rmse:.2f}")


MAE: 1.56
RMSE: 3.17


Considering 48h:

In [22]:
df_merged = df_merged[df_merged['CHARTTIME'] <= df_merged['INTIME'] + pd.Timedelta(hours=72)]
print(df_merged.columns)

df = df[df['ICUSTAY_ID'].isin(df_merged['ICUSTAY_ID'].unique())]
print(len(df))

Index(['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'ITEMID', 'CHARTTIME', 'VALUE',
       'INTIME'],
      dtype='object')
23413


In [23]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = df.drop(columns=['LOS', 'ICUSTAY_ID']) 
y = df['LOS']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [24]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"MAE: {mae:.2f}")   
print(f"RMSE: {rmse:.2f}")


MAE: 1.56
RMSE: 3.17


The results were quite similar across the various time windows we tested; however, we found that the 24-hour window yielded the worst results.